In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import os
import warnings

# 파일 불러와서 열이름 바꾸기
korean, long_term_frgn, short_term_frgn, resident, card_cnt, card_amt, waste_cnt
(이거 예시는 card_amt임)

In [2]:
df = pd.read_csv("files\\data_preprocessing\\korean_preprocessing.csv", encoding='cp949')

# 시계열 피처단위로 변경
df['base_date'] = pd.to_datetime(df['base_date'])

# base_date를 index로 설정
df.index = df['base_date']
df.columns = ['base_date', 'emd_cd', 'korean']

In [3]:
# 필요 없는 열 제거
data = df.drop(['base_date'], 1)  
data

,emd_cd,korean
base_date,,
2018-01-31,50110250,7.209306e+06
2018-02-28,50110250,5.579125e+06
2018-03-31,50110250,7.615021e+06
2018-04-30,50110250,8.498349e+06
2018-05-31,50110250,8.916782e+06
...,...,...
2021-02-28,50130620,4.321265e+06
2021-03-31,50130620,4.744309e+06
2021-04-30,50130620,5.236706e+06


In [4]:
# 한 행정동의 행만 추출
sub_area = data['emd_cd'] == 50110540  ##각행정동코드

# base_date, card_amt 만 갖는 df 생성
sub_df = data[sub_area]
sub_df = sub_df.drop('emd_cd', 1)
sub_df

,korean
base_date,
2018-01-31,2.359747e+07
2018-02-28,2.068806e+07
2018-03-31,2.325933e+07
2018-04-30,2.217670e+07
2018-05-31,2.289490e+07
2018-06-30,2.194265e+07
2018-07-31,2.331979e+07
2018-08-31,2.279283e+07
2018-09-30,2.190337e+07


In [5]:
# series 형으로 변환
sub_df.to_dict('series')

{'korean': base_date
 2018-01-31    2.359747e+07
 2018-02-28    2.068806e+07
 2018-03-31    2.325933e+07
 2018-04-30    2.217670e+07
 2018-05-31    2.289490e+07
 2018-06-30    2.194265e+07
 2018-07-31    2.331979e+07
 2018-08-31    2.279283e+07
 2018-09-30    2.190337e+07
 2018-10-31    2.202675e+07
 2018-11-30    2.180541e+07
 2018-12-31    2.259948e+07
 2019-01-31    2.356383e+07
 2019-02-28    2.097794e+07
 2019-03-31    2.257608e+07
 2019-04-30    2.102220e+07
 2019-05-31    2.211557e+07
 2019-06-30    2.113815e+07
 2019-07-31    2.281548e+07
 2019-08-31    2.185760e+07
 2019-09-30    2.066435e+07
 2019-10-31    2.148564e+07
 2019-11-30    2.142363e+07
 2019-12-31    2.141119e+07
 2020-01-31    2.231640e+07
 2020-02-29    1.991689e+07
 2020-03-31    1.956194e+07
 2020-04-30    1.854631e+07
 2020-05-31    1.955537e+07
 2020-06-30    2.014526e+07
 2020-07-31    2.187425e+07
 2020-08-31    2.045621e+07
 2020-09-30    1.929410e+07
 2020-10-31    2.090732e+07
 2020-11-30    2.113889e+07

# ARIMA

In [6]:
# ARIMA에 적합한 float로 바꾸는 과정
sub_df_float = sub_df[:].astype(np.float)

In [7]:
# ADF 검정 - 정상성 확인하는 검정 (귀무 : 정상성 만족 x, 대립 : 정상성 만족)
from statsmodels.tsa.stattools import adfuller

result = adfuller(sub_df)
print('ADF Statistic : %f' % result[0])
print('p-value : %f' % result[1])
print('Critical Values :')
for key, value in result[4].items():
    print('\t%s: %3f' % (key, value))

ADF Statistic : -3.726442
p-value : 0.003756
Critical Values :
	1%: -3.600983
	5%: -2.935135
	10%: -2.605963


In [8]:
from pmdarima.arima import auto_arima

model_arima= auto_arima(sub_df,trace=True, error_action='ignore',suppress_warnings=True,stepwise=False,seasonal=True)

model_arima.fit(sub_df)

 ARIMA(0,1,0)(0,0,0)[1] intercept   : AIC=1274.499, Time=0.01 sec
 ARIMA(0,1,1)(0,0,0)[1] intercept   : AIC=1269.013, Time=0.10 sec
 ARIMA(0,1,2)(0,0,0)[1] intercept   : AIC=1268.192, Time=0.05 sec
 ARIMA(0,1,3)(0,0,0)[1] intercept   : AIC=1273.321, Time=0.11 sec
 ARIMA(0,1,4)(0,0,0)[1] intercept   : AIC=1276.048, Time=0.06 sec
 ARIMA(0,1,5)(0,0,0)[1] intercept   : AIC=1272.273, Time=0.14 sec
 ARIMA(1,1,0)(0,0,0)[1] intercept   : AIC=1266.218, Time=0.04 sec
 ARIMA(1,1,1)(0,0,0)[1] intercept   : AIC=1263.160, Time=0.08 sec
 ARIMA(1,1,2)(0,0,0)[1] intercept   : AIC=1262.954, Time=0.17 sec
 ARIMA(1,1,3)(0,0,0)[1] intercept   : AIC=1264.027, Time=0.20 sec
 ARIMA(1,1,4)(0,0,0)[1] intercept   : AIC=1267.106, Time=0.27 sec
 ARIMA(2,1,0)(0,0,0)[1] intercept   : AIC=1264.738, Time=0.05 sec
 ARIMA(2,1,1)(0,0,0)[1] intercept   : AIC=1262.141, Time=0.17 sec
 ARIMA(2,1,2)(0,0,0)[1] intercept   : AIC=1262.926, Time=0.27 sec
 ARIMA(2,1,3)(0,0,0)[1] intercept   : AIC=1265.390, Time=0.33 sec
 ARIMA(3,1

ARIMA(order=(2, 1, 1), scoring_args={}, seasonal_order=(0, 0, 0, 1),
      suppress_warnings=True)

In [9]:
# ARIMA 모델 
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm

# (AR=2, 차분=1, MA=2) 파라미터로 ARIMA 모델을 학습합니다.
model = ARIMA(sub_df.korean.values, order=(2,1,1))   

#trend : constant를 가지고 있는지, c - constant / nc - no constant
#disp : 수렴 정보를 나타냄
model_fit = model.fit(trend='c', full_output=True, disp=True)
print(model_fit.summary())

C:\Users\soeun\anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)


                             ARIMA Model Results                              
Dep. Variable:                    D.y   No. Observations:                   41
Model:                 ARIMA(2, 1, 1)   Log Likelihood                -623.208
Method:                       css-mle   S.D. of innovations         924199.419
Date:                Sat, 11 Sep 2021   AIC                           1256.416
Time:                        01:54:34   BIC                           1264.984
Sample:                             1   HQIC                          1259.536
                                                                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const      -7.956e+04   1.23e+04     -6.469      0.000   -1.04e+05   -5.55e+04
ar.L1.D.y      0.0322      0.158      0.204      0.838      -0.277       0.341
ar.L2.D.y      0.0098      0.168      0.059      0.9

C:\Users\soeun\anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)


# 예측하기

In [10]:
# 2단위 이후의 예측결과
fore = model_fit.forecast(steps=2)
print(fore)

(array([19556448.01114447, 19470913.45082793]), array([924199.41851305, 924679.65009923]), array([[17745050.43632603, 21367845.5859629 ],
       [17658574.63939633, 21283252.26225952]]))
